# Api Nifi

## Librerias

In [ ]:
from urllib import response
import requests
import json
import urllib3
from IPython.display import display
import time
import sys
import warnings
import cx_Oracle
##desactivamos los mensajes de peligro
import paramiko
from prettytable import PrettyTable
from prettytable import MSWORD_FRIENDLY
from prettytable.colortable import  ColorTable, Themes
urllib3.disable_warnings()

## Datos de Accesos Globales

### NiFi

In [ ]:
USERNAME = "EXD21089"
PASSWORD = "Lalito7284"
URL = "https://cdf01it-01atl.claro.amx:8443/nifi-api" 
# USERNAME = "a3b9dbc7-485d-4813-aa4d-5c0db8b14f03"
# PASSWORD = "r0jH5WuH6/JTGyCyuCC4MKxKGQG9jkiK"
# URL = "https://127.0.0.1:8443/nifi-api"

### Acceso a la base de datos

In [ ]:
### username y password ponerle db
dbusername = 'dstreamsdb' ##usuario
dbpassword = 'claro2015' ##contraseña
dbdsn = 'exa1-scan.claro.amx/RAC8.world' ##host
dbport = 1512 ##puerto de acceso
encoding = 'UTF-8'

### SSH

In [ ]:
host="vertientes6.claro.amx"
sshuser="streamsadmin"
sshpass="Vates2015"

### Acceso al token de la api

In [ ]:
#Obtención del token de nifi
def obtenerToken(URL):
    ingreso = "/access/token?username={}&password={}".format(USERNAME, PASSWORD)
    HeadersSinJson = {'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8', 'User-Agent': 'PostmanRuntime/7.29.0'}
    response = requests.post(URL+ingreso, headers=HeadersSinJson, verify = False)        
    if response.status_code == 201 or response.status_code == 200:
       #devuelve el token en byte, no como string
        base = response.content
        token = base.decode("utf-8")
        return token
    else:
        print(response.status_code)
    
    #Recorremos el archivo con el escenario de prueba

TokenNifi = obtenerToken(URL)

## Consulta de base datos

In [ ]:
def oraclesSentence(dbSentece):
    ### error no poner sus columnas, te va mostrar una sola columna
    """
          ** DESA **
        Host: exa1-scan.claro.amx
        Nombre de Servicio: RAC8.WORLD
        User: dstreamsdb
        Pass: claro2015
    """
    connection = None
    try: 
        connection = cx_Oracle.connect(dbusername,dbpassword,dbdsn,encoding=encoding)
        ### obtenemos los campos para tener las columnas de las tablas, separando los campos de select por espacio que hay entre ambos 
        ### y lo cierra el espacio entre el ultimo campo y el from
        campos = dbSentece.split(" ")[1]
        tabla = dbSentece.split(" ")[3]
        ### en caso de un where ingresemos con #el valor string para cambiarlos por '' #
        new_s = dbSentece.replace("'","'")
        if campos == '*':
            columnas = []
            cursor = connection.cursor()
            for row in cursor.execute("SELECT column_name FROM all_tab_columns WHERE table_name = '"+tabla+"'"):                
                columnas.append(row[0])            
            cursor.close()            
            y = PrettyTable(columnas)
        else:
            ### Convertimos una lista la cadena de texto
            output=campos.split(',')  
            y = PrettyTable(output)
            
        cursor = connection.cursor()
        for fila in cursor.execute(new_s):
                y.add_row(fila)
                print()
        cursor.close()
        print(y.get_string(title="Resultado de "+tabla))
    except cx_Oracle.Error as error:
        print(error)
    finally:
    # release the connection
        if connection:
            connection.close()   

## Diccionarios y tablas

### Diccionarios y lector de los procesadores

In [ ]:
""" 
    @uthor: Leonardo Alverto Rébola
    Description: Creacion de una clase de diccionario
"""
class CustomDict(dict):
    #__init___function
    def __init__(self):
        self = dict()
        
    #Function to add key:value
    def add(self, key, value):
        self[key] = value

        
class TableDict(dict):
    #__init___function
    def __init__(self):
        self = dict()
        
    #Function to add key:value
    def add(self, key, value):
        self[key] = value
        
class GroupsDict(dict):
    #__init___function
    def __init__(self):
        self = dict()
        
    #Function to add key:value
    def add(self, key, value):
        self[key] = value
        
#instancia del dict
dict_proccessors = CustomDict()
    
#desglozamos y buscamos los datos de los proccessors
def loadProccessors(pg):
    if type(pg) == int:
        return False
    else:
        proccessors = pg['processGroupFlow']['flow']['processors']
        for proccessor in proccessors:
            ##error add() takes 3 positional arguments but 4 were given puede ser por que dentro revision tiene dos valores
            dict_proccessors.add(proccessor['component']['name'], proccessor['component']['id'])
        return dict_proccessors

### Diccionario y lector de conectores

In [ ]:
class Connection:
    def __init__(self,id, name, sourceId, sourceName, destinationId, destinationName):
        self.id = id
        self.name = name
        self.sourceId = sourceId
        self.sourceName = sourceName
        self.destinationId = destinationId
        self.destinationName = destinationName
    
    def __str__(self):
        return "id:{},name:{}, sourceId:{}, sourceName:{}, destinationId:{}, destinationName:{}".format(self.id, self.name, self.sourceId ,self.sourceName , self.destinationId, self.destinationName)

dict_connections = CustomDict()
def loadConnections(pg):
    if type(pg) == int:
        return False
    else:
        connections = pg['processGroupFlow']['flow']['connections']
        for conection in connections:
            dict_connections.add(conection['component']['name'],
                                 Connection(
                                    conection['component']['id'],
                                    conection['component']['name'],
                                    conection['component']['source']['id'],
                                    conection['component']['source']['name'],
                                    conection['component']['destination']['id'],
                                    conection['component']['destination']['name']
                                 )
                                )
        return dict_connections

### Diccionario para tabla de procesadores

In [ ]:
class TableProcessor:
    def __init__(self,name, tipo , status, flowFilesIn, flowFilesOut, taskCount):
        self.name = name
        self.tipo = tipo
        self.status = status
        self.flowFilesIn = flowFilesIn
        self.flowFilesOut = flowFilesOut
        self.taskCount = taskCount
    
    def __str__(self):
        return "name:{},tipo:{}, status:{}, flowFilesIn:{}, flowFilesOut:{}, taskCount:{}".format(self.name, self.tipo, self.status ,self.flowFilesIn , self.flowFilesOut, self.taskCount)


### Diccionario para tabla de Connection

In [ ]:
class TableConnection:
    def __init__(self, name, sourceName, destinationName, flowFilesQueued, queued, queuedCount, flowFilesIn, aceptados, flowFilesOut, procesados):
        ## aceptado = input
        ## procesadors = output
        self.name = name
        self.sourceName = sourceName
        self.destinationName = destinationName
        self.flowFilesQueued = flowFilesQueued
        self.queued = queued
        self.queuedCount = queuedCount
        self.flowFilesIn = flowFilesIn
        self.aceptado = aceptados
        self.flowFilesOut = flowFilesOut
        self.procesados =procesados
    
    def __str__(self):
        return "name:{}, sourceName:{}, destinationName:{}, flowFilesQueued:{}, queued:{}, queuedCount:{}, flowFilesIn:{}, aceptados:{}, flowFilesOut:{}, procesados:{}".format(self.name, self.sourceName, self.destinationName, self.flowFilesQueued, self.queued, self.queuedCount, self.flowFilesIn, self.aceptados, self.flowFilesOut, self.procesados)

### Diccionario  y lector de ProcessGroup anidado

In [ ]:
class processGroupDict(dict):
    #__init___function
    def __init__(self):
        self = dict()
        
    #Function to add key:value
    def add(self, key, value):
        self[key] = value

class processGroup:
    def __init__(self, name, uuid):
        self.name = name,
        self.uuid = uuid
    
    def __str__(self):
        return "name:{} ,uuid:{}".format(self.name, self.uuid)
    
dict_proccessgroup = processGroupDict()

def loadProcessgroups(pg):
    if type(pg) == int:
        return False
    else:
        proccessgroups = pg['processGroupFlow']['flow']['processGroups']
        for proccessgroup in proccessgroups:
            ##error add() takes 3 positional arguments but 4 were given puede ser por que dentro revision tiene dos valores
            dict_proccessgroup.add(proccessgroup['component']['name'], proccessgroup['component']['id'])                    
        return dict_proccessgroup

### SSH

In [ ]:
def SshParamiko(command):
    try:
        ssh = paramiko.SSHClient()
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        ssh.connect(host, username=sshuser, password=sshpass)
        sftp = ssh.open_sftp()
        tdin, stdout, stderr = ssh.exec_command(command)
        output = stdout.read()
        ssh.close()
        respuesta = output.decode('utf-8')
        return respuesta
    except Exception as e:
        return str(e)
    

## Funciones proccessors

### GET Processors groups

In [ ]:
#obtencion de datos del proccess groups
dict_tablep = TableDict()
def getProccessGroups(TokenNifi, id):
    flowProccessGroupsURI = URL + "/flow/process-groups/"
    #URL al process-group
    url =flowProccessGroupsURI+id
    
    #encabezado
    headers = {'Content-Type':'application/json', 'Authorization':'Bearer ' + TokenNifi}
    
    #almacena el request
    payload ={'uiOnly':True}
    response = requests.get(url, headers=headers, verify=False, params=payload)
    
    if response.status_code == 201 or response.status_code == 200: 
        response_json = response.json()
        return response_json
    else:
        return response.status_code

### Play Processgroup

In [ ]:
def playProcessGroup(uuid):
    #agregar url
    EnviarToken = "/flow/process-groups/"+ uuid
    #encabezado
    headers = {'Accept': '*/*','Content-Type':'application/json;charset=UTF-8','User-Agent': 'PostmanRuntime/7.29.0', 'Authorization':'Bearer '+TokenNifi}
    #almacena el request
    data={'id': uuid,"state": "RUNNING"}
    response = requests.put(URL+EnviarToken,headers=headers,verify=False, json=data)
    if response.status_code == 201 or response.status_code == 200:
        response_json = response.json()
        return response_json
    else:
        display(response.text)
        return response.status_code  
   
    
    

### Stop Processgroup

In [ ]:
def stopProcessGroup(uuid):
    #agregar url
    EnviarToken = "/flow/process-groups/"+ uuid
    #encabezado
    headers = {'Accept': '*/*','Content-Type':'application/json;charset=UTF-8','User-Agent': 'PostmanRuntime/7.29.0', 'Authorization':'Bearer '+TokenNifi}
    #almacena el request
    data={'id': uuid,"state": "STOPPED"}
    response = requests.put(URL+EnviarToken,headers=headers,verify=False, json=data)
    #Prueba
    if response.status_code == 201 or response.status_code == 200:
        response_json = response.json()
        return response_json
    else:
        display(response.text)
        return response.status_code  
   
    

### Variables Globales

#### obtencion del las variables globales

In [ ]:
def GetGlobalRevision(id, variable):
    #agregar url
    EnviarToken = "/process-groups/"+id+"/variable-registry"
    #encabezado
    headers = {'Accept': '*/*','Content-Type':'application/json;charset=UTF-8','User-Agent': 'PostmanRuntime/7.29.0', 'Authorization':'Bearer '+TokenNifi}
    #splits
    name = variable.split(":")[0]
    valor = variable.split(":")[1]
    #almacena el request
    response = requests.get(URL+EnviarToken,headers=headers,verify=False)
    #Prueba
    if response.status_code == 201 or response.status_code == 200:
        response_json = response.json()
        variables = response_json['processGroupRevision']
        res = EditarGlobalVariable(variables,id,name,valor)
        if res == True:
            return True
        else:
            return res
        
    else:
        display(response.text)
        return response.text

#### edición de las variables globales

In [ ]:
def EditarGlobalVariable(variables,id,name,valor):
    EnviarToken = "/process-groups/"+id+"/variable-registry"
    #encabezado
    headers = {'Accept': '*/*','Content-Type':'application/json;charset=UTF-8','User-Agent': 'PostmanRuntime/7.29.0', 'Authorization':'Bearer '+TokenNifi}
    #almacena el request
    variables={"processGroupRevision":variables,"variableRegistry":{"variables": [{"variable": {"name": name,"value": valor}}],"processGroupId": id}}
    response = requests.put(URL+EnviarToken,headers=headers,verify=False, json=variables)
    #Prueba
    if response.status_code == 201 or response.status_code == 200:
        response_json = response.json()
        return True
    else:
        return response.text
         

### GET Proccessor

In [ ]:
# Traer los datos del proccesor
def GetProccessor(id):
    #agregar al url
    EnviarToken = "/processors/"+id
    #encabezado
    headers = {'Content-Type':'application/json', 'Authorization':'Bearer ' + TokenNifi}
    #almacena el requestzzz
    response = requests.get(URL+EnviarToken,headers=headers,verify=False)
    #Prueba
    if response.status_code == 201 or response.status_code == 200:
        response_json = response.json()
        return response_json['revision']
    else:
        display(response.text)

### PLAY Proccessor

In [ ]:
dict_tableP = CustomDict()
def PlayProccessor(uuid,revision):
    #agregar al url
    EnviarToken = "/processors/"+uuid+"/run-status"
    #encabezado
    headers = {'Accept': '*/*','Content-Type':'application/json;charset=UTF-8','User-Agent': 'PostmanRuntime/7.29.0', 'Authorization':'Bearer '+TokenNifi}
    #almacena el request
    data={'revision': revision,"state": "RUNNING"}
    response = requests.put(URL+EnviarToken,headers=headers,verify=False, json=data)
    #Prueba
    if response.status_code == 201 or response.status_code == 200:
        response_json = response.json()
        procesador = response_json["status"]
        
        dict_tableP.add(procesador["aggregateSnapshot"]['name']+"-Play",
                        TableProcessor(
                            procesador["aggregateSnapshot"]['name'],
                            procesador["aggregateSnapshot"]['type'],
                            procesador["aggregateSnapshot"]['runStatus'],
                            procesador["aggregateSnapshot"]['flowFilesIn'],
                            procesador["aggregateSnapshot"]['flowFilesOut'],
                            procesador["aggregateSnapshot"]['taskCount']
                        )
                    )
 
        return True
    else:
        return False

### STOP Proccessor

In [ ]:
def StopProccessor(uuid,stoped):
    #agregar al url
    EnviarToken = "/processors/"+uuid+"/run-status"
    #encabezado
    headers = {'Accept': '*/*','Content-Type':'application/json;charset=UTF-8','User-Agent': 'PostmanRuntime/7.29.0', 'Authorization':'Bearer '+TokenNifi}
    #almacena el request
    data={"revision": stoped,"state": "STOPPED"}
    response = requests.put(URL+EnviarToken,headers=headers,verify=False, json=data)
    #Prueba
    if response.status_code == 201 or response.status_code == 200: 
        response_json = response.json()
        procesador = response_json["status"]
        
        dict_tableP.add(procesador["aggregateSnapshot"]['name']+"-Stop",
                        TableProcessor(
                            procesador["aggregateSnapshot"]['name'],
                            procesador["aggregateSnapshot"]['type'],
                            procesador["aggregateSnapshot"]['runStatus'],
                            procesador["aggregateSnapshot"]['flowFilesIn'],
                            procesador["aggregateSnapshot"]['flowFilesOut'],
                            procesador["aggregateSnapshot"]['taskCount']
                        )
                    )
        return True
    else:
        return False

### CLEAR STATE Proccessor

In [ ]:
def ClearProccessor(id):
    #aregar al url
    EnviarToken = "/processors/"+id+"/state/clear-requests"
    #encabezado
    headers = {'Accept': '*/*','Content-Type':'application/json;charset=UTF-8','User-Agent': 'PostmanRuntime/7.29.0', 'Authorization':'Bearer '+TokenNifi}
    #almacena el request
    response = requests.post(URL+EnviarToken,headers=headers,verify=False)
    #Pueba
    if response.status_code == 201 or response.status_code == 200:            
        print("Datos del State limpiados")
        return True
    else:
        return False

## Connecctions

### Get connecction

In [ ]:
dict_tableC = TableDict()
def GetConnecction(id):
    #agregar al url
    EnviarToken = "/connections/"+id
    #encabezado
    headers = {'Accept': '*/*','Content-Type':'application/json;charset=UTF-8','User-Agent': 'PostmanRuntime/7.29.0', 'Authorization':'Bearer '+TokenNifi}
    #almacena el request
    response = requests.get(URL+EnviarToken,headers=headers,verify=False)
    #Prueba
    if response.status_code == 201 or response.status_code == 200: 
        response_json = response.json()
        connections = response_json["status"]
        version = response_json["component"]['source']["running"]
        dict_tableC.add(connections["aggregateSnapshot"]['name']+str(version),
                        TableConnection(
                            connections["aggregateSnapshot"]['name'],
                            connections["aggregateSnapshot"]['sourceName'],
                            connections["aggregateSnapshot"]['destinationName'],
                            connections["aggregateSnapshot"]['flowFilesQueued'],
                            connections["aggregateSnapshot"]['queued'],
                            connections["aggregateSnapshot"]['queuedCount'],
                            connections["aggregateSnapshot"]['flowFilesIn'],
                            connections["aggregateSnapshot"]['input'],
                            connections["aggregateSnapshot"]['flowFilesOut'],
                            connections["aggregateSnapshot"]['output']
                        )
                    )
        return True
    else:
        return False

## Generadores de tablas

### Tabla de procesadores

In [ ]:
def informeTablaProcessor():
    diccionarios = dict(dict_tableP)
    x = PrettyTable(["Nombre", "Tipo", "Estatus", "flowFiles aceptados", "flowFiles transferido", "Tareas Ejecutadas"])
    ### alinamiento centro
    x.align = "c"
    ### style
    x.set_style(MSWORD_FRIENDLY)
    ###preserve_internal_border
    x.preserve_internal_border  = True
    for diccionario in diccionarios.keys():
        connection = dict_tableP.get(diccionario, "Processor no encontrado")
        conecetor = connection.__dict__
        x.add_row([conecetor["name"],conecetor["tipo"], conecetor["status"], conecetor["flowFilesIn"],conecetor["flowFilesOut"],conecetor["taskCount"]])
   
    ###auto indexado
    # x.add_autoindex('Pasos')
    print(x.get_string(title="Procesadores"))

### Tabla de conectores

In [ ]:
def informeTablaConnector():
    diccionaries = dict(dict_tableC)
    y = PrettyTable(["Nombre", "Origen", "Destino","flowFiles en espera", "flowFiles aceptados","flowFiles transferido"])
    ### alinamiento centro
    y.align = "c"
    ### style
    y.set_style(MSWORD_FRIENDLY)
    ###preserve_internal_border
    y.preserve_internal_border  = True
    for diccies in diccionaries.keys():
        connection = dict_tableC.get(diccies, "Conector no encontrado")
        conecetor = connection.__dict__
        y.add_row([conecetor["name"],conecetor["sourceName"],conecetor["destinationName"],conecetor["queued"],conecetor["aceptado"],conecetor["procesados"]])
    ###auto indexado
    # y.add_autoindex('Pasos')
    print(y.get_string(title="Conectores"))


## Lectura del caso de prueba

In [ ]:
def getIDfromLine(line):
    return line.split("'")[1]

def readFileInput():
    inputfile = open("ussd-sms-ar-mix.txt",'r')
    lines = inputfile.readlines()
    for line in lines:
        readEscenario(line)
        
def readEscenario(line):
    if "FEATURE" in line.upper():
        display("FEATURE "+getIDfromLine(line))
    elif "SCENARIO" in line.upper():
        display("SCENARIO "+getIDfromLine(line))
    elif "DATAFLOW" in line.upper():
               #UUID del proccessgroups ingresado en el archivo de txt
        uuid = getIDfromLine(line)
        print("Dataflow "+uuid)
               ##obtenemos los datos del proccessgroup
        if getProccessGroups(TokenNifi, uuid) == False:            
            warnings.filterwarnings("ignore") 
            sys.exit('exiting...')
        else:
            loadProccessors(getProccessGroups(TokenNifi, uuid))
            loadConnections(getProccessGroups(TokenNifi, uuid))
            loadProcessgroups(getProccessGroups(TokenNifi, uuid))
        ### en get proccessGroups si sale 400 cerrar el ciclo con exit 
######################################################################################################################################################################################################        
    elif "INICIAR PROCESADOR" in line.upper():
        name = getIDfromLine(line)
        uuid = dict_proccessors.get(name, "Processor no encontrado")
        revision = GetProccessor(uuid)
        status_code = PlayProccessor(uuid,revision)
        if status_code == True:
            print("\n")
            print(">>>>> PLAY >>>>> " + name + " con UUID " + uuid)
            del revision
            del uuid
            del name
            time.sleep(10)
        else:
            try:
                warnings.filterwarnings("ignore")            
                sys.exit('exiting...')            
            except TypeError:
                print("Los datos son incompatibles")
 ######################################################################################################################################################################################################         
    elif "DETENER PROCESADOR" in line.upper():
        name = getIDfromLine(line)
        uuid = dict_proccessors.get(name, "Processor no encontrado")        
        stoped = GetProccessor(uuid)
        status_code = StopProccessor(uuid,stoped)
        if status_code == True:
            print("\n")
            print(">>>>> STOP >>>>> " + name + " con UUID " + uuid)
            time.sleep(3)
        else:
            try:
                warnings.filterwarnings("ignore")            
                sys.exit('exiting...')            
            except TypeError:
                print("Los datos son incompatibles")
########################################################################################################################################################################################################        
    elif "INGRESAR PROCESSGROUP" in line.upper():        
        name = getIDfromLine(line)
        uuid = dict_proccessgroup.get(name, "ProcessGroup no encontrado")
        global UUID 
        UUID = uuid
        loadProccessors(getProccessGroups(TokenNifi, uuid))
        loadConnections(getProccessGroups(TokenNifi, uuid))
        loadProcessgroups(getProccessGroups(TokenNifi, uuid))
        print(">>>>> ProcessGroup >>>>> " + name +"  "+ uuid)
        
        
########################################################################################################################################################################################################        
    elif "CHEQUEAR  CONEXION" in line.upper():        
        name = getIDfromLine(line)        
        connection = dict_connections.get(name, "Conector no encontrado")
        if not type(connection) == str:
            conecetor = connection.__dict__
            flowfile = GetConnecction(conecetor['id'])
            if flowfile == True:
                time.sleep(3)
            else:
                try:
                    warnings.filterwarnings("ignore")            
                    sys.exit('exiting...')            
                except TypeError:
                    print("Los datos son incompatibles")
        else:
            try:
                warnings.filterwarnings("ignore")            
                sys.exit('exiting...')            
            except TypeError:
                print("Los datos son incompatibles")
            
########################################################################################################################################################################################################        
    elif "LIMPIAR STAGE" in line.upper():        
        name = getIDfromLine(line)
        uuid = dict_proccessors.get(name, "Processor no encontrado")        
        stoped = GetProccessor(uuid)
        status_code = ClearProccessor(uuid)
        if status_code == True:
            print("\n")
            print(">>>>> Clear >>>>> " + name + " con UUID " + uuid)
            time.sleep(3)
        else:
            try:
                warnings.filterwarnings("ignore")            
                sys.exit('exiting...')            
            except TypeError:
                print("Los datos son incompatibles")
########################################################################################################################################################################################################
    elif "IMPRIMIR" in line.upper():        
        Impresion = getIDfromLine(line)
        if "ALL" in Impresion.upper():
            informeTablaProcessor()
            print("\n")
            informeTablaConnector()
            print("\n")
        elif "PROCESSOR" in  Impresion.upper():
            informeTablaProcessor()
            print("\n")
        elif "CONNECTOR" in Impresion.upper():
            informeTablaConnector()            
            print("\n")
        else:            
            print("Error opcion inexistente")            
            print("\n")
            
##########################################################################################################################################################################################################
    elif "CONSULTA SQL" in line.upper():        
        dbSentece = getIDfromLine(line)        
        resultado = oraclesSentence(dbSentece)
        display(resultado)
###########################################################################################################################################################################################################
    elif "EDITAR VARIABLE GLOBAL" in line.upper():
        name_and_value = getIDfromLine(line)        
        cambios = GetGlobalRevision(UUID, name_and_value)
        if cambios == True:
            print("\n")
            print("El Valor ingresado para la variable se ha cambiado con exito")            
            print("\n")
        else:
            print(cambios)
###########################################################################################################################################################################################################
    elif "COMANDO SSH" in line.upper():
        cmd = getIDfromLine(line)
        respuesta = SshParamiko(cmd)
        print(respuesta)

###########################################################################################################################################################################################################
    elif "TIEMPO DE ESPERA" in line.upper():
        number = getIDfromLine(line)
        print("Inicia el tiempo de pausa "+ number+ " segundo")
        espera= int(number)
        time.sleep(espera)
        print("termino el tiempo de pausa")
########################################################################################################################################################################################################        
    elif "INICIAR PROCESSGROUP" in line.upper():        
        name = getIDfromLine(line)
        uuid = dict_proccessgroup.get(name, "ProcessGroup no encontrado")
        print(uuid)
        playProcessGroup(uuid)
        print("\n")
        print(">>>>> PLAY >>>>> " + name + " con UUID " + uuid)
########################################################################################################################################################################################################        
    elif "DETENER PROCESSGROUP" in line.upper():        
        name = getIDfromLine(line)
        uuid = dict_proccessgroup.get(name, "ProcessGroup no encontrado")
        stopProcessGroup(uuid)
        print("\n")
        print(">>>>> STOP >>>>> " + name + " con UUID " + uuid)        
        
########################################################################################################################################################################################################        
 
readFileInput()      

# Fin